In [ ]:
import os
from src.utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology, grainGenerate
from src.cfg import CfgAnglesNames, CfgBeamsNames, CfgDataset
from skimage import transform
from skimage import io

In [ ]:
folder_path = '../data/dataset_cleaned'
new_folder_path = '../data/dataset_left_orig'

folders = os.listdir(folder_path)

# создает папку для обработанных фотографий
if not os.path.exists(new_folder_path):
    os.mkdir(new_folder_path)

# создает папку для каждого класса фотографий
for folder in folders:
    if not os.path.exists(new_folder_path + '/' + folder):
        os.mkdir(new_folder_path + '/' + folder)

for i, folder in enumerate(folders):
    images_names = os.listdir(folder_path + '/' + folder)
    for i, name in enumerate(images_names):
        if 'hdr' not in name:
            image = io.imread(folder_path + '/' + folder + '/' + name)
            image_combined = grainPreprocess.combine(image, h=144)

            io.imsave(new_folder_path + '/' + folder + '/' + name , image_combined)


In [ ]:
folder_path = '../data/dataset_left_orig'
new_folder_path = '../data/dataset_left_4x'

folders = os.listdir(folder_path)

# создает папку для обработанных фотографий
if not os.path.exists(new_folder_path):
    os.mkdir(new_folder_path)

# создает папку для каждого класса фотографий
for folder in folders:
    if not os.path.exists(new_folder_path + '/' + folder):
        os.mkdir(new_folder_path + '/' + folder)

for i, folder in enumerate(folders):
    images_names = os.listdir(folder_path + '/' + folder)
    for i, name in enumerate(images_names):
        if 'hdr' not in name:
            image = io.imread(folder_path + '/' + folder + '/' + name)
            # image_combined = grainPreprocess.combine(image, h=144,k=1)
            height, width = image.shape
            img1 = image[:height // 2, :width // 2]
            img2 = image[:height // 2, width // 2:]
            img3 = image[height // 2:, :width // 2]
            img4 = image[height // 2:, width // 2:]
            # image_resized = transform.resize(image_combined, (512,512),preserve_range=True).astype(np.uint8)

            # io.imsave(new_folder_path + '/' + folder + '/' + name , image_combined)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_1' + name[-5:], img1)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_2' + name[-5:], img2)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_3' + name[-5:], img3)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_4' + name[-5:], img4)

In [ ]:
folder_path = '../data/dataset_left_4x'
new_folder_path = '../data/dataset_left_4x_360'

folders = os.listdir(folder_path)

# создает папку для обработанных фотографий
if not os.path.exists(new_folder_path):
    os.mkdir(new_folder_path)

# создает папку для каждого класса фотографий
for folder in folders:
    if not os.path.exists(new_folder_path + '/' + folder):
        os.mkdir(new_folder_path + '/' + folder)

for i, folder in enumerate(folders):
    images_names = os.listdir(folder_path + '/' + folder)
    for i, name in enumerate(images_names):
        if 'hdr' not in name:
            image = io.imread(folder_path + '/' + folder + '/' + name)
            # image_combined = grainPreprocess.combine(image, h=144,k=1)
            height, width = image.shape
            img1 = image
            img2 = transform.rotate(image,90)
            img3 = transform.rotate(image,180)
            img4 = transform.rotate(image,270)
            # image_resized = transform.resize(image_combined, (512,512),preserve_range=True).astype(np.uint8)

            # io.imsave(new_folder_path + '/' + folder + '/' + name , image_combined)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_angle_0' + name[-5:], img1)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_angle_90' + name[-5:], img2)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_angle_180' + name[-5:], img3)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_angle_270' + name[-5:], img4)